In [ ]:
    import pprint
    
    pp = pprint.PrettyPrinter(indent=4)    

# Mesos/Marathon Python

## Deploying MiniMesos

### Requirements

* Docker
* marathon-python (through pip install)

#### For Fedora/RHEL/CentOS users

MiniMesos is not compatible with the docker package provided by Red Hat. You'll have to use the packages provided by Docker, Inc.

```sh
sudo dnf config-manager \
    --add-repo \
    https://download.docker.com/linux/fedora/docker-ce.repo
sudo dnf remove docker
sudo dnf install docker-ce
sudo systemctl restart docker
```


### Install MiniMesos

```sh
curl -sSL https://minimesos.org/install | sh # This installs MiniMesos in $HOME/.minimesos/bin
export PATH=$PATH:$HOME/.minimesos/bin
```

## Now ready to play with MiniMesos
First let's view our minimesos environment from the command-line

In [7]:
! minimesos init

Initialized minimesosFile in this directory


In [9]:
! minimesos --debug up

Initialized debug logging
Executing up command
Creating Mesos cluster
Creating cluster processes
Validating cluster processes
Connecting cluster processes
Writing cluster ID 2635860384 to /home/hguemar/Labs/Orchestration-ApacheMesos.py/.minimesos/minimesos.cluster
Cluster 2635860384 - start
Failed to run command 'up'. null


In [4]:
! minimesos info

Minimesos cluster is running: 2969741956
Mesos version: 1.0.0
export MINIMESOS_NETWORK_GATEWAY=172.17.0.1
export MINIMESOS_AGENT=http://172.17.0.6:5051; export MINIMESOS_AGENT_IP=172.17.0.6
export MINIMESOS_ZOOKEEPER=zk://172.17.0.3:2181/mesos; export MINIMESOS_ZOOKEEPER_IP=172.17.0.3
export MINIMESOS_MARATHON=http://172.17.0.5:8080; export MINIMESOS_MARATHON_IP=172.17.0.5
export MINIMESOS_CONSUL=http://172.17.0.7:8500; export MINIMESOS_CONSUL_IP=172.17.0.7
export MINIMESOS_MASTER=http://172.17.0.4:5050; export MINIMESOS_MASTER_IP=172.17.0.4


In [5]:
! minimesos state

{
  "git_sha": "c9b70582e9fccab8f6863b0bd3a812b5969a8c24",
  "leader": "master@172.17.0.4:5050",
  "cluster": "Change Cluster Name in minimesosFile file",
  "slaves": [{
    "used_resources": {
      "disk": 0,
      "mem": 128,
      "cpus": 1,
      "gpus": 0,
      "ports": "[31443-31443]"
    },
    "hostname": "172.17.0.6",
    "registered_time": 1.49495118974121E9,
    "offered_resources": {
      "disk": 0,
      "mem": 0,
      "cpus": 0,
      "gpus": 0
    },
    "unreserved_resources": {
      "disk": 2000,
      "mem": 1024,
      "cpus": 4,
      "gpus": 0,
      "ports": "[31000-32000]"
    },
    "resources": {
      "disk": 2000,
      "mem": 1024,
      "cpus": 4,
      "gpus": 0,
      "ports": "[31000-32000]"
    },
    "active": true,
    "pid": "slave(1)@172.17.0.6:5051",
    "attributes": {},
    "id": "87badae9-48f3-446e-b693-b4fcc0499322-S0",
    "reserved_resources": {},
    "version": "1.0.0"
  }],
  "unregistered_frameworks": [],
  "activated_slaves": 1,
  "f

In [1]:
! minimesos ps

FRAMEWORK            TASK                 STATE                PORT                
marathon             weave-scope          TASK_RUNNING         31443               


# Play with marathon

The following snippet will load minimesos configuration into python global scope as variables.

In [54]:
import io
import os
import re
import subprocess

EXPATTERN = r'export (?P<var>\w+)=(?P<val>[\w\.:/]+)'

def load_info():
    def parse_output(out):
        return re.findall(EXPATTERN, out)
                
    out = subprocess.run(['minimesos', 'info'], 
                         stdout=subprocess.PIPE,
                        encoding='utf8').stdout
    res = parse_output(out)
    g = globals()
    for var, val in res:
        g[var] = val

load_info()

In [55]:
print("You can visit Marathon web UI at this URL: {}".format(MINIMESOS_MARATHON))

You can visit Marathon web UI at this URL: http://172.17.0.6:8080


# Experiments

## Marathon-python

Let's try to interact with Marathon using the python API


In [140]:
from marathon import MarathonClient, MarathonApp
from time import sleep

m = MarathonClient(MINIMESOS_MARATHON)
m.get_info()
m.list_apps()


[MarathonApp::/weave-scope, MarathonApp::/sleepyapp]

In [144]:
m.create_app('sleepyapp', MarathonApp(cmd='sleep 100', mem=16, cpus=1))
app = m.get_app('sleepyapp')
sleep(30)
app.instances
m.scale_app('sleepyapp', instances=2)
sleep(30)
app.instances
m.list_tasks()
sleep(30)
m.delete_app('sleepyapp', force=True)
sleep(30)
m.list_tasks()

[MarathonTask::weave-scope.725ef4bf-3b1f-11e7-a27e-0242ac110006]

In [144]:
m.create_app('nginx', MarathonApp(cmd='sleep 100', mem=16, cpus=1))
app = m.get_app('sleepyapp')
sleep(30)
app.instances
m.scale_app('sleepyapp', instances=2)
sleep(30)
app.instances
m.list_tasks()
sleep(30)
m.delete_app('sleepyapp', force=True)
sleep(30)
m.list_tasks()

[MarathonTask::weave-scope.725ef4bf-3b1f-11e7-a27e-0242ac110006]

## REST API

Let's deploy nginx using curl

In [67]:
! curl -X POST $MINIMESOS_MARATHON/v2/apps -d @nginx.json -H "Content-type: application/json" | jq


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1040    0   887  100   153   6995   1206 --:--:-- --:--:-- --:--:--  7039
{
  "id": "/nginx",
  "cmd": null,
  "args": null,
  "user": null,
  "env": {},
  "instances": 1,
  "cpus": 0.1,
  "mem": 128,
  "disk": 0,
  "gpus": 0,
  "executor": "",
  "constraints": [],
  "uris": [],
  "fetch": [],
  "storeUrls": [],
  "backoffSeconds": 1,
  "backoffFactor": 1.15,
  "maxLaunchDelaySeconds": 3600,
  "container": {
    "type": "MESOS",
    "volumes": [],
    "docker": {
      "image": "library/nginx",
      "credential": null,
      "forcePullImage": false
    }
  },
  "healthChecks": [],
  "readinessChecks": [],
  "dependencies": [],
  "upgradeStrategy": {
    "minimumHealthCapacity": 1,
    "maximumOverCapacity": 1
  },
  "labels": {},
  "acceptedResourceRoles": null,
  "ipAddress": null,
  "version": "2017-05-17T17:04:20.462Z",
 

In [58]:
! curl $MINIMESOS_MARATHON/v2/apps -H "Content-type: application/json" | jq


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2187    0  2187    0     0   391k      0 --:--:-- --:--:-- --:--:--  427k
{
  "apps": [
    {
      "id": "/weave-scope",
      "cmd": null,
      "args": [
        "--probe.docker",
        "true"
      ],
      "user": null,
      "env": {
        "CHECKPOINT_DISABLE": ""
      },
      "instances": 1,
      "cpus": 1,
      "mem": 128,
      "disk": 0,
      "gpus": 0,
      "executor": "",
      "constraints": [
        [
          "hostname",
          "UNIQUE"
        ]
      ],
      "uris": [],
      "fetch": [],
      "storeUrls": [],
      "backoffSeconds": 1,
      "backoffFactor": 1.15,
      "maxLaunchDelaySeconds": 3600,
      "container": {
        "type": "DOCKER",
        "volumes": [
          {
            "containerPath": "/var/run/docker.sock",
            "hostPath": "/var/run/docker.sock",
            "

In [107]:
! echo "Your nginx runs at this url http://"$(curl $MINIMESOS_MARATHON/v2/apps/nginx -H "Content-type: application/json" | jq -r ".app.tasks[0].ipAddresses[0].ipAddress")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1319    0  1319    0     0   255k      0 --:--:-- --:--:-- --:--:--  322k
Your nginx runs at this url http://172.17.0.7


Let's try to scale out our nginx

In [108]:
! curl -X PUT $MINIMESOS_MARATHON/v2/apps/nginx -d @nginx-scale-out.json -H "Content-type: application/json"


{"version":"2017-05-17T17:50:34.042Z","deploymentId":"b526602e-f846-4859-a332-d76bbf347f60"}

Now scale down

In [109]:
! curl -X PUT $MINIMESOS_MARATHON/v2/apps/nginx -d @nginx-scale-down.json -H "Content-type: application/json"


{"version":"2017-05-17T17:51:58.699Z","deploymentId":"19d21d07-ae01-4323-b67e-dbbb330739b4"}

Destroy the app nginx now.

In [110]:
! curl -X DELETE $MINIMESOS_MARATHON/v2/apps/nginx -H "Content-type: application/json" 


{"version":"2017-05-17T17:52:17.379Z","deploymentId":"19f2f2ab-2145-4567-a0f2-3781f04f6226"}

Now, let's check if our application is deleted.